In [1]:
from tsfresh_feature_extraction import *

In [2]:
set_seed(612)

In [3]:
train_extracted_minmax_large=np.load("./numpy/train_extracted_minmax_train_0322.npy")
test_extracted_minmax_large=np.load("./numpy/test_extracted_minmax_train_0322.npy")
device=CFG.device
CFG.epoches=350
longlist=np.concatenate((train_extracted_minmax_large,test_extracted_minmax_large),axis=\
                        0)[:,np.random.permutation(100)]
train_extracted_minmax_large,test_extracted_minmax_large=np.vsplit(longlist,np.array([24020]))
device=CFG.device
train_dataset_expend_encoded = AircraftDataset_expend_feature_extraction(df_train, train_label, 
                                                                         torch.FloatTensor(train_extracted_minmax_large),False)
train_encoded_loader = DataLoader(
    train_dataset_expend_encoded,
    batch_size=CFG.batch_size,
    shuffle=True,
    drop_last=True,
)
test_dataset = AircraftDataset_no_expend_feature_extraction(df_test, test_label, torch.FloatTensor(test_extracted_minmax_large))
test_encoded_loader = DataLoader(
    test_dataset,
    batch_size=1,
    shuffle=False)
for train_data in train_encoded_loader:None
for test_data in test_encoded_loader:None

In [202]:
path = DATA_PATH+'RUL.csv'
RUL_frame = pd.read_csv(path, header=None)
RUL = RUL_frame.values[:, 0]
RUL_y=pd.Series(RUL)
RUL_y.index+=1

In [107]:
lifetime_list=[]
for i in range(len(test_dataset)):
    lifetime=test_dataset[i]["lifetime"].item()
    lifetime_list.append(lifetime)
lifetime_list=np.array(lifetime_list)

In [564]:
aml_model_2=AML_model2(100,100,100,128,2,5,shared=False, attention=True,expand=False).to(device)
aml_model_2.load_state_dict(torch.load("./model_checkpoints_at/model_his_aml.pt"))

<All keys matched successfully>

In [534]:
h_idx=96 #[41, 44, 51, 53, 60, 64, 96]

In [535]:
aml_model_2(test_dataset[h_idx]["input"].unsqueeze(0).to(device))[0].item(),RUL[h_idx]

(274.92938232421875, 123)

In [536]:
test_dataset[h_idx]["input"]

tensor([-0.0623, -0.0900,  0.1538, -0.4582, -0.4992, -0.0873, -0.8177, -0.2934,
         0.0929, -0.0150,  0.1789,  0.1471,  0.4016, -0.7453, -0.7023, -0.1114,
        -0.3668,  0.9637,  0.1138,  0.0892,  0.0621,  0.0892,  0.5966,  0.5645,
        -0.7715,  0.1752,  0.2803, -0.2493,  0.5713, -0.9678,  0.0260,  0.0355,
        -0.7691, -1.0000, -0.8891,  0.1287,  0.3501, -1.0000, -0.1225,  0.1171,
         0.1066, -0.7691, -0.9920, -0.6655, -0.2780,  0.2255,  0.3367, -0.1162,
        -0.7381,  0.5237, -0.9294,  0.0997, -0.2855,  0.5353, -0.6576,  0.1394,
        -0.5156, -0.9098,  0.1789, -0.1870,  0.3267, -0.1621,  0.2796, -0.5556,
         0.3234,  0.0355, -0.9346, -0.2780, -0.2774, -0.4968, -0.8160, -0.9317,
        -0.5075, -0.6179,  0.4105, -0.9933,  0.5870, -0.8668,  0.5878, -0.9185,
         0.4518,  0.1193, -0.4783, -0.2345, -0.8702,  0.0109,  0.0057, -0.8968,
         0.1618, -0.0436, -0.3134,  0.2057,  0.5969,  0.2302,  0.9789,  0.1066,
         0.2329,  0.4244,  0.1585,  0.33

In [537]:
distance_list=torch.zeros(24020)
for idx in range(24020):
    x = train_dataset_expend_encoded[idx]["input"]
    y = test_dataset[h_idx]["input"]
    distance = F.pairwise_distance(x, y, p=2)
    #distance_list.append(distance)
    distance_list[idx] = 1/distance

In [538]:
#diag=torch.exp(distance_list).unsqueeze(-1) 
diag=distance_list.unsqueeze(-1) 

In [539]:
# diag[torch.where(diag < torch.quantile(diag, 0.8, dim=0,
#                                                 keepdim=False,
#                                                interpolation='nearest'))] = 0

In [540]:
diag_reg=diag / torch.sum(diag)

In [541]:
test_44=torch.sum(torch.FloatTensor(train_extracted_minmax_large)*diag_reg,dim=0)

In [542]:
test_44

tensor([ 6.6492e-02,  9.0698e-02,  9.2041e-02, -7.4392e-02, -9.2405e-02,
         4.1875e-02, -5.0341e-01, -6.9835e-02,  2.3363e-01,  2.0949e-01,
         2.0916e-01,  1.7804e-01,  2.9650e-01, -1.0636e-01, -5.9507e-01,
        -3.6142e-04, -3.4769e-01,  1.5915e-01,  1.3851e-01,  2.0414e-01,
         1.4602e-01,  2.1593e-01,  4.2202e-01,  4.3226e-01, -4.2381e-01,
         2.7669e-01,  4.3138e-01,  6.2192e-03,  3.2987e-01, -7.9167e-01,
         2.0402e-01,  1.3386e-01, -5.0117e-01, -1.2422e-02, -7.5528e-01,
         2.0311e-01,  4.0049e-01, -4.7184e-02,  9.4762e-03,  1.3503e-01,
         8.4575e-02, -5.0117e-01, -9.2445e-01, -1.2785e-01,  8.0379e-02,
         2.7635e-01,  2.8753e-01,  3.2550e-02, -1.4574e-01,  4.1814e-01,
        -7.9833e-01,  2.5698e-01, -8.1084e-02,  4.0197e-01, -3.5175e-01,
         2.6123e-01, -2.7435e-02, -7.5184e-01,  2.0916e-01,  5.2908e-02,
         2.4976e-01,  2.4421e-01,  2.4361e-01, -5.6726e-01,  2.2131e-01,
         1.3386e-01, -6.3399e-01,  8.0379e-02, -3.9

In [543]:
aml_model_2(test_44.unsqueeze(0).to(device))[0].item(),RUL[h_idx]

(88.93962860107422, 123)

# 特征增强函数

In [552]:
def feature_enhance(instance_train,instance_test):
    #instance_test_enhanced = instance_test
    feature_enhanced_list=[]
    for test_idx in range(len(instance_test)):
        distance_list=torch.zeros(len(instance_train))
        y = instance_test[test_idx]["input"]
        for train_idx in range(len(instance_train)):
            x = instance_train[train_idx]["input"]
            distance = F.pairwise_distance(x, y, p=2)
            distance_list[train_idx] = 1/distance
        #diag=torch.exp(distance_list).unsqueeze(-1) 
        diag=distance_list.unsqueeze(-1)
#         diag[torch.where(diag < torch.quantile(diag, 0.5, dim=0,
#                                             keepdim=False,
#                                            interpolation='nearest'))] = 0
        diag_reg=diag / torch.sum(diag)
        if instance_test[test_idx]["lifetime"]<113:
            feature_enhanced_list.append(torch.sum(instance_train.all_feature_list*diag_reg,dim=0))
        else:
            feature_enhanced_list.append(y)
    return feature_enhanced_list    # 
                

In [553]:
#test_dataset_enhanced = feature_enhance(train_dataset_expend_encoded,test_dataset) #非常耗时的
feature_enhanced_list = feature_enhance(train_dataset_expend_encoded,test_dataset)

In [554]:
test_dataset_enhanced = AircraftDataset_no_expend_feature_extraction(df_test, 
                                                            test_label, 
                                                            feature_enhanced_list)

In [555]:
test_encoded_enhanced_loader = DataLoader(
    test_dataset_enhanced,
    batch_size=1,
    shuffle=False)

In [565]:
pred_2=[]
for idx,test_data in enumerate(test_encoded_enhanced_loader):
    pred_2_q=aml_model_2(test_data["input"].to(device))[0].item()
    pred_2.append(pred_2_q)
#RUL
path = DATA_PATH+'RUL.csv'
RUL_frame = pd.read_csv(path, header=None)
RUL = RUL_frame.values[:, 0]
np.sum(np.ceil((abs(np.array(pred_2)-RUL))))#目测结果

1752.0

In [566]:
get_Accuracy(np.array(pred_2), RUL, lifetime_list)

matrix([[2.50000000e+01, 1.99635571e-02, 5.89919628e-03],
        [5.00000000e+01, 2.19749984e-02, 4.33802086e-03],
        [7.50000000e+01, 4.87135107e-02, 1.22891108e-02],
        [1.00000000e+02, 4.94432974e-02, 9.29555482e-03],
        [1.25000000e+02, 6.34846845e-02, 8.76490113e-03],
        [1.50000000e+02, 7.89745741e-02, 9.03031720e-03]])

In [559]:
np.where(np.ceil((abs(np.array(pred_2)-RUL)))>70)

(array([], dtype=int64),)

In [558]:
np.save("./numpy/20220401feature_enhanced_list_frac.npy",np.array([feature_enhanced_list[i].numpy() for i in range(100)]))

# 特征增强2

In [372]:
h_idx=27

In [373]:
aml_model_2(test_dataset[h_idx]["input"].unsqueeze(0).to(device))[0].item(),RUL[h_idx]

(218.00733947753906, 144)

In [374]:
y0 = torch.zeros(100)
length = len(train_dataset_expend_encoded)
for sensor in range(100):
    feature_train = torch.FloatTensor(train_extracted_minmax_large)[:,sensor]
    distance_list=torch.zeros(length)
    y = test_dataset[h_idx]["input"][sensor]
    for idx in range(length):
        x = train_dataset_expend_encoded[idx]["input"][sensor]
        distance = -F.pairwise_distance(x, y, p=2)
        #distance_list.append(distance)
        distance_list[idx] = distance
    diag=torch.exp(distance_list)
#         diag[torch.where(diag < torch.quantile(diag, 0.6, dim=0,
#                                             keepdim=False,
#                                            interpolation='nearest'))] = 0
    diag_reg=diag / torch.sum(diag)
    y0[sensor] = torch.inner(feature_train,diag_reg)

In [375]:
aml_model_2(y0.unsqueeze(0).to(device))[0].item(),RUL[h_idx]

(163.63601684570312, 144)

In [480]:
def feature_enhance3(instance_train,instance_test):
    #instance_test_enhanced = instance_test
    feature_enhanced_list=[]
    for test_idx in range(len(instance_test)):
        if instance_test[test_idx]["lifetime"]<100:
            y0 = torch.zeros(100)
            length = len(instance_train)
            for sensor in range(100):
                feature_train = torch.FloatTensor(instance_train.all_feature_list)[:,sensor]
                distance_list=torch.zeros(length)
                y = instance_test[test_idx]["input"][sensor]
                for idx in range(length):
                    x = instance_train[idx]["input"][sensor]
                    distance = -F.pairwise_distance(x, y, p=2)
                    #distance_list.append(distance)
                    distance_list[idx] = distance
                #diag=torch.exp(distance_list)
                diag=distance_list
            #         diag[torch.where(diag < torch.quantile(diag, 0.6, dim=0,
            #                                             keepdim=False,
            #                                            interpolation='nearest'))] = 0
                diag_reg=diag / torch.sum(diag)
                y0[sensor] = torch.inner(feature_train,diag_reg)
            feature_enhanced_list.append(y0)
        
#         diag[torch.where(diag < torch.quantile(diag, 0.5, dim=0,
#                                             keepdim=False,
#                                            interpolation='nearest'))] = 0
#         diag_reg=diag / torch.sum(diag)
        #instance_test_enhanced[test_idx]["input"] = torch.sum(instance_train.all_feature_list*diag_reg,dim=0)

        else:
            feature_enhanced_list.append(instance_test[test_idx]["input"])
    return feature_enhanced_list     

In [481]:
feature_enhanced_list_2 = feature_enhance3(train_dataset_expend_encoded,test_dataset)

KeyboardInterrupt: 

In [ ]:
np.save("./numpy/20220401feature_enhanced_list_sper.npy",np.array([feature_enhanced_list_2[i].numpy() for i in range(100)]))

In [ ]:
nb3 = np.load("./numpy/20220401feature_enhanced_list_sper.npy") 

In [ ]:
test_dataset_enhanced = AircraftDataset_no_expend_feature_extraction(df_test, 
                                                            test_label, 
                                                            nb2)
test_encoded_enhanced_loader = DataLoader(
    test_dataset_enhanced,
    batch_size=1,
    shuffle=False)
pred_2=[]
for idx,test_data in enumerate(test_encoded_enhanced_loader):
    pred_2_q=aml_model_2(test_data["input"].to(device))[0].item()
    pred_2.append(pred_2_q)
#RUL
path = DATA_PATH+'RUL.csv'
RUL_frame = pd.read_csv(path, header=None)
RUL = RUL_frame.values[:, 0]
np.sum(np.ceil((abs(np.array(pred_2)-RUL))))#目测结果

In [ ]:
get_Accuracy(np.array(pred_2), RUL, lifetime_list)